In [3]:
import pandas as pd

In [4]:
old_grid = pd.read_csv('moebert_distil_grid.csv', sep=';')

In [5]:
new_grid_ = old_grid[old_grid['experiment_number'] <= 100]

In [6]:
ds_list = ['rte', 'cola','mrpc', 'sst2', 'qqp', 'mnli', 'qnli']
ds_list_exp = []
for a in ds_list:
    for _ in range(100):
        ds_list_exp.append(a)

In [7]:
# duplicate new_grid_ for each dataset
new_grid = pd.concat([new_grid_]*len(ds_list), ignore_index=True)
# add dataset column, duplicate sequentially like rte rte rte ... 
new_grid['dataset'] = ds_list_exp
new_grid['perm_epoch'] = 0


In [8]:
best_params_dict = {
    'rte': {'batch_size': 8, 'weight_decay': 0.01, 'gamma': 0.1},
    'cola': {'batch_size': 8, 'weight_decay': 0.1, 'gamma': 0.01},
    'mrpc': {'batch_size': 8, 'weight_decay': 0.01, 'gamma': 0.1},
    'sst2': {'batch_size': 16, 'weight_decay': 0.01, 'gamma': 1},
    'qqp': {'batch_size': 16, 'weight_decay': 0.01, 'gamma': 1},
    'mnli': {'batch_size': 16, 'weight_decay': 0, 'gamma': 0.01},
    'qnli': {'batch_size': 16, 'weight_decay': 0, 'gamma': 1},
}

perm_epoch = [i for i in range(1, 11)]

# Apply best params to new_grid
for ds in ds_list:
    new_grid.loc[new_grid['dataset'] == ds, 'batch_size'] = best_params_dict[ds]['batch_size']
    new_grid.loc[new_grid['dataset'] == ds, 'weight_decay'] = best_params_dict[ds]['weight_decay']
    new_grid.loc[new_grid['dataset'] == ds, 'gamma'] = best_params_dict[ds]['gamma']

new_grid['perm_epoch'] = perm_epoch * 70


In [9]:
new_grid

,experiment_number,batch_size,learning_rate,weight_decay,gamma,entropy,distill,dataset,perm_epoch
0,1,8,0.00001,0.01,0.1,0.05,1,rte,1
1,2,8,0.00002,0.01,0.1,0.10,2,rte,2
2,3,8,0.00003,0.01,0.1,0.50,3,rte,3
3,4,8,0.00004,0.01,0.1,1.00,4,rte,4
4,5,8,0.00001,0.01,0.1,5.00,5,rte,5
...,...,...,...,...,...,...,...,...,...
695,96,16,0.00004,0.00,1.0,0.05,4,qnli,6
696,97,16,0.00001,0.00,1.0,0.05,5,qnli,7
697,98,16,0.00002,0.00,1.0,0.10,1,qnli,8
698,99,16,0.00003,0.00,1.0,0.50,3,qnli,9


In [10]:
new_grid.to_csv('moebert_perm_distil_grid.csv', sep=';', index=True)

In [19]:
hyper_params_all = pd.read_csv(
    "moebert_perm_distil_grid.csv",
    sep=";",
    index_col=0,
    dtype={
        "experiment_number": int,
        "batch_size": int,
        "learning_rate": float,
        "weight_decay": float,
        "entropy": float,
        "gamma": float,
        "distill": float,
        "dataset": str,
        "perm_epoch": float,
    },
    )

hyper_params = hyper_params_all[hyper_params_all.dataset == 'qnli']
hyper_params.drop(columns=["dataset", "experiment_number"], inplace=True)

# set experiment_num as index
hyper_params.index = range(1, 101)


/state/partition1/llgrid/pkg/anaconda/anaconda3-2021b/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [20]:
hyper_params

,batch_size,learning_rate,weight_decay,gamma,entropy,distill,perm_epoch
1,16,0.00001,0.0,1.0,0.05,1.0,1.0
2,16,0.00002,0.0,1.0,0.10,2.0,2.0
3,16,0.00003,0.0,1.0,0.50,3.0,3.0
4,16,0.00004,0.0,1.0,1.00,4.0,4.0
5,16,0.00001,0.0,1.0,5.00,5.0,5.0
...,...,...,...,...,...,...,...
96,16,0.00004,0.0,1.0,0.05,4.0,6.0
97,16,0.00001,0.0,1.0,0.05,5.0,7.0
98,16,0.00002,0.0,1.0,0.10,1.0,8.0
99,16,0.00003,0.0,1.0,0.50,3.0,9.0
